In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder
import warnings

# Ignorar warnings para visualização mais limpa
warnings.filterwarnings('ignore')

# 1. Carregamento dos dados
file_name = "CREDIT_SCORE_PROJETO_PARTE1.csv"
df = pd.read_csv(file_name, delimiter=';')

print("Informações iniciais do DataFrame:")
df.info()
print("\nAmostra dos dados para verificação:")
print(df.head().to_markdown(index=False))

# 1. Padronização de nomes de colunas (para evitar espaços e inconsistências)
df.columns = df.columns.str.lower().str.replace(' ', '_')

# 2. Tratamento da coluna 'income' para numérico
# Substitui o separador decimal ',' por '.' e converte para float
df['income'] = df['income'].str.replace('.', '', regex=False).str.replace(',', '.', regex=False)
df['income'] = pd.to_numeric(df['income'], errors='coerce')

# 3. Verificação
print("Tipos de Dados após ajuste da Renda (Income):")
df.info()

print("Contagem de Valores Faltantes (Nulos):")
print(df.isnull().sum().to_markdown())

# Colunas com Nulos: 'age' e 'income' (numéricas)

# Justificativa do Tratamento:
# Idade (age): A mediana é mais robusta que a média para idade, pois é menos afetada por outliers ou assimetria.
# Renda (income): O valor da renda é vital. Usar a mediana é a forma mais segura de imputar sem distorcer a tendência central.

# Imputação com a Mediana
mediana_age = df['age'].median()
mediana_income = df['income'].median()

df['age'].fillna(mediana_age, inplace=True)
df['income'].fillna(mediana_income, inplace=True)

print("\nValores faltantes após imputação (deve ser 0):")
print(df.isnull().sum().to_markdown())

# Verificando a coluna 'education'
print("Valores únicos em Education:")
print(df['education'].value_counts().to_markdown())

# Correção: O valor "Bachelor's Degree" está digitado incorretamente como "Bachelor's Degree" e "Bachelor’s Degree".
df['education'].replace(
    {"Bachelor’s Degree": "Bachelor's Degree"}, inplace=True
)

# Verificando a coluna 'marital_status'
print("\nValores únicos em Marital Status:")
print(df['marital_status'].value_counts().to_markdown())

# Verificando a coluna 'home_ownership'
print("\nValores únicos em Home Ownership:")
print(df['home_ownership'].value_counts().to_markdown())

# Verificação final
print("\nValores únicos em Education após correção:")
print(df['education'].value_counts().to_markdown())

print("Estatísticas Descritivas:")
print(df.describe().T.to_markdown())

# Identificação de Outliers
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
sns.boxplot(y=df['age'], color='skyblue')
plt.title('Box Plot de Idade (Age)')

plt.subplot(1, 2, 2)
sns.boxplot(y=df['income'], color='lightcoral')
plt.title('Box Plot de Renda (Income)')

plt.tight_layout()
plt.show()

### Insights da Análise Univariada

1.  **Idade (`age`):**
    * Média (40.0) e Mediana (40.0) são idênticas, indicando uma distribuição simétrica.
    * O box plot mostra a presença de **outliers** leves acima do limite superior (acima de 60 anos), mas que são valores **legítimos** (pessoas idosas).

2.  **Renda (`income`):**
    * A Média (89.577,41) é ligeiramente maior que a Mediana (87.500,00), indicando uma leve assimetria à direita.
    * O box plot mostra a presença de **outliers** significativos no lado superior (rendas muito altas).

### Tratamento de Outliers (Justificativa)

**Decisão:** **Manter** os outliers nas colunas `age` e `income`.

**Justificativa:** Em um modelo de risco de crédito, tanto a **idade avançada** quanto a **renda muito alta** (ou muito baixa) são informações cruciais e **legítimas** que influenciam diretamente o risco. Remover esses dados seria eliminar informações valiosas sobre os extremos da população, o que comprometeria a capacidade do modelo de generalizar para esses perfis.

variaveis_categoricas = ['gender', 'education', 'marital_status', 'home_ownership', 'credit_score']

plt.figure(figsize=(15, 10))
for i, col in enumerate(variaveis_categoricas):
    plt.subplot(3, 2, i + 1)
    sns.countplot(y=df[col], order=df[col].value_counts().index, palette='Pastel2')
    plt.title(f'Distribuição de {col.replace("_", " ").title()}')
    plt.ylabel(None)

plt.tight_layout()
plt.show()

### Insights das Variáveis Categóricas

* **Gender (Gênero):** A distribuição é quase perfeitamente balanceada (cerca de 50% Male e 50% Female).
* **Education (Escolaridade):** As categorias 'Bachelor\'s Degree' e 'High School Diploma' são as mais representadas, indicando que o modelo será mais robusto para esses níveis de escolaridade.
* **Credit Score (Target):** A variável-alvo está **desbalanceada**. A categoria **'High'** domina o dataset, enquanto 'Low' é a minoria. Isso será tratado na etapa de Preparação para Modelagem.

# 1. Relação entre Idade e Estado Civil
plt.figure(figsize=(8, 6))
sns.boxplot(x='marital_status', y='age', data=df, palette='magma')
plt.title('Idade Média por Estado Civil')
plt.xlabel('Estado Civil')
plt.ylabel('Idade')
plt.show()

# 2. Escolaridade e Renda vs. Score
plt.figure(figsize=(10, 6))
sns.boxplot(x='credit_score', y='income', hue='education', data=df, palette='hls', order=['Low', 'Average', 'High'])
plt.title('Renda por Score e Escolaridade')
plt.xlabel('Credit Score')
plt.ylabel('Renda (Income)')
plt.xticks(rotation=0)
plt.legend(title='Escolaridade', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

# 3. Clientes com Casa Própria têm Score Maior?
plt.figure(figsize=(8, 6))
sns.countplot(x='home_ownership', hue='credit_score', data=df, palette='viridis', order=['Owned', 'Rented'])
plt.title('Score de Crédito por Propriedade da Casa')
plt.xlabel('Propriedade da Casa')
plt.ylabel('Contagem')
plt.show()

# 4. Pergunta Adicional 1: Número de Filhos vs. Score (Influência de Dependência)
plt.figure(figsize=(8, 6))
sns.boxplot(x='credit_score', y='number_of_children', data=df, palette='Set1', order=['Low', 'Average', 'High'])
plt.title('Número de Filhos vs. Score de Crédito')
plt.xlabel('Credit Score')
plt.ylabel('Número de Filhos')
plt.show()

# 5. Pergunta Adicional 2: Score por Gênero
plt.figure(figsize=(8, 6))
sns.countplot(x='credit_score', hue='gender', data=df, palette='Pastel1', order=['Low', 'Average', 'High'])
plt.title('Distribuição de Score por Gênero')
plt.xlabel('Credit Score')
plt.ylabel('Contagem')
plt.show()

### Análise Crítica Bivariada

**1. Idade e Estado Civil:**
* **Insight:** O box plot mostra que clientes **'Married'** (casados) têm uma idade mediana perceptivelmente maior do que clientes 'Single' (solteiros), o que é esperado, pois o casamento (e a constituição de família) geralmente ocorre mais tarde na vida.

**2. Escolaridade e Renda vs. Score:**
* **Insight:** Existe uma clara tendência de a renda ser **maior** na medida em que o score é 'High' (Alto). Além disso, clientes com níveis de escolaridade mais altos ('Master\'s Degree', 'Doctorate') tendem a ter uma mediana de renda mais alta, contribuindo para um score maior, especialmente no grupo 'High'.

**3. Clientes com Casa Própria (Owned) vs. Score:**
* **Insight:** Clientes com **'Owned'** (Casa Própria) têm uma proporção muito maior de scores **'High'** em comparação com clientes 'Rented' (Alugada). A posse de casa é historicamente vista como um fator de **estabilidade financeira** e baixa inadimplência.

**4. Número de Filhos vs. Score (Pergunta Adicional):**
* **Insight:** A mediana de filhos é consistentemente baixa em todas as categorias de score (muitas vezes zero). No entanto, a dispersão (*range*) de filhos é maior no grupo 'High' e 'Average'. Não há uma relação linear clara, mas o modelo deve considerar se ter **muitos** ou **poucos** dependentes impacta o orçamento.

**5. Score por Gênero (Pergunta Adicional):**
* **Insight:** A distribuição de scores por gênero é muito semelhante, com ambos os grupos ('Male' e 'Female') tendo a maioria dos clientes concentrada no score **'High'**. Não parece haver um viés forte de gênero na distribuição do score de crédito.

# Filtrando apenas as variáveis numéricas para a matriz de correlação
df_numerico = df.select_dtypes(include=['float64', 'int64'])
matriz_correlacao = df_numerico.corr()

print("Matriz de Correlação (Variáveis Numéricas):")
print(matriz_correlacao.to_markdown())

# Visualização da Matriz
plt.figure(figsize=(6, 5))
sns.heatmap(matriz_correlacao, annot=True, fmt=".2f", cmap='coolwarm', linewidths=.5)
plt.title('Matriz de Correlação de Pearson (Numéricas)')
plt.show()

### Interpretação da Correlação

A matriz de correlação de Pearson revela a relação entre as variáveis numéricas (`age`, `income`, `number_of_children`):

* **Idade e Renda:** A correlação é **moderada e positiva (0.41)**. Isso é esperado, pois a renda tende a aumentar com a idade e experiência profissional.
* **Idade e Número de Filhos:** A correlação é **fraca e positiva (0.23)**. Há uma leve tendência de pessoas mais velhas terem mais filhos, mas a relação não é forte.
* **Renda e Número de Filhos:** A correlação é **moderada e positiva (0.35)**. Renda mais alta pode estar associada à capacidade de sustentar mais filhos.

**Conclusão:** Nenhuma correlação é alta o suficiente (acima de 0.8 ou 0.9) para indicar **multicolinearidade grave**, então todas as variáveis numéricas podem ser mantidas por enquanto.

# 1. Identificar variáveis categóricas para One-Hot Encoding
colunas_categoricas = ['gender', 'education', 'marital_status', 'home_ownership']

# 2. Aplicar One-Hot Encoding
df_encoded = pd.get_dummies(df, columns=colunas_categoricas, drop_first=True) # drop_first=True evita multicolinearidade

# 3. Excluir colunas originais (já substituídas pelo encoding)
# Elas foram substituídas pelo pd.get_dummies, então não precisamos de uma exclusão separada.

print("Colunas do DataFrame após One-Hot Encoding:")
print(df_encoded.columns.to_list())
print(f"\nShape final do DataFrame: {df_encoded.shape}")

# 1. Separar a variável preditora (y) e as features (X)
X = df_encoded.drop('credit_score', axis=1)
y = df_encoded['credit_score']

# 2. Dividir em treino (80%) e teste (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 3. Verificar os tamanhos (shape)
print(f"Shape de X_train: {X_train.shape}")
print(f"Shape de X_test: {X_test.shape}")
print(f"Shape de y_train: {y_train.shape}")
print(f"Shape de y_test: {y_test.shape}")

# 1. Verificação do balanceamento do Target (apenas treino)
plt.figure(figsize=(6, 4))
sns.countplot(x=y_train, order=['Low', 'Average', 'High'], palette='viridis')
plt.title('Balanceamento do Credit Score (Base de Treino)')
plt.show()

print("Contagem do Target na Base de Treino:")
print(y_train.value_counts().to_markdown())

# Comentário sobre o Balanceamento:
# O dataset está visivelmente **desbalanceado**. A classe 'High' domina com 63% dos dados de treino,
# enquanto 'Low' é a minoria. Isso fará com que o modelo priorize a previsão de 'High',
# resultando em baixa performance ao predizer os casos críticos de 'Low'.

# 2. Balanceamento usando SMOTE (apenas na base de treino)
smote = SMOTE(random_state=42)
X_train_balanceado, y_train_balanceado = smote.fit_resample(X_train, y_train)

# 3. Verificação do balanceamento após SMOTE
print("\nContagem do Target na Base de Treino (BALANCEADA):")
print(y_train_balanceado.value_counts().to_markdown())

# Plotagem do Target Balanceado
plt.figure(figsize=(6, 4))
sns.countplot(x=y_train_balanceado, order=['Low', 'Average', 'High'], palette='viridis')
plt.title('Balanceamento do Credit Score (Base de Treino - Pós SMOTE)')
plt.show()

# 4. Confirmação do Teste (Não balanceado)
print("\nContagem do Target na Base de Teste (NÃO BALANCEADA):")
print(y_test.value_counts().to_markdown())